In [8]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import pickle
import threshold
import find_lane_lines as finder
import curvature_measurement as cm
import draw_lane as dl
import transform_perspective as trp
import warp

M = pickle.load(open( "perspectiveTransformMatrix.p", "rb" ))
Minv = pickle.load(open( "oppositePerspectiveTransformMatrix.p", "rb" ))

dist_pickle = pickle.load( open( "calibration.p", "rb" ) )
mtx = dist_pickle[0]
dist = dist_pickle[1]

In [2]:
def show_gray(images):
    for im in images:
        plt.figure()
        plt.imshow(im, cmap='gray')

def show(images):
    for im in images:
        plt.figure()
        plt.imshow(im)

In [ ]:


test_images = glob.glob('test_images/*')
images = []
for i, filename in enumerate(test_images):
    img = cv2.imread(filename)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    images.append(dst)
    #plt.figure()
    #plt.imshow(dst)
#plt.show()

In [ ]:
thresholded_images = []
for image in images:
    thr = threshold.process_image(image)
    thresholded_images.append(thr)
    #show_gray([thr])
#plt.show()

In [ ]:
M = pickle.load(open( "perspectiveTransformMatrix.p", "rb" ))
import transform_perspective as trp
transformed_images = []
for image in thresholded_images:
    imshape = image.shape
    trf = trp.transform(image, M)
    transformed_images.append(trf)
    show_gray([trf])
plt.show()

In [ ]:
measurements = []
for image in transformed_images:
    ploty, left_fitx, right_fitx = finder.find(image)
    measurements.append([ploty, left_fitx, right_fitx])
plt.show()

In [ ]:
for m in measurements:
    print(cm.measure(m[0], m[1], m[2]))

In [ ]:
Minv = pickle.load(open( "oppositePerspectiveTransformMatrix.p", "rb" ))
i = 0
image = images[i]
warped = transformed_images[i]
m = measurements[i]
ploty, left_fitx, right_fitx = m[0], m[1], m[2]

result = dl.draw(image, warped, ploty, left_fitx, right_fitx, Minv)
plt.imshow(result)
plt.show()

In [6]:
def process(image):
    dst = cv2.undistort(image, mtx, dist, None, mtx)
    transformed = trp.transform(dst, M)
    thresholded = threshold.process_image(transformed)
    warped = warp.process(thresholded)
    imshape = image.shape
    ploty, left_fitx, right_fitx, ret = finder.find(warped)
    if ret == False:
        return image
    measurements = cm.measure(ploty, left_fitx, right_fitx)
    result = dl.draw(image, warped, ploty, left_fitx, right_fitx, Minv)
    return transformed

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

output = 'out1.mp4'
clip1 = VideoFileClip("project_video.mp4")
#clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(process) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video out1.mp4
[MoviePy] Writing video out1.mp4


 36%|███▌      | 454/1261 [02:42<04:47,  2.80it/s]

In [ ]:
'''''
    a = (210,imshape[0]-50)
    b = ((imshape[1]/2)-180, 520)
    c = ((imshape[1]/2)+180, 520)
    d = (imshape[1]-210,imshape[0]-50)
    
    a = (190,imshape[0])
    b = ((imshape[1]/2)-35, 440)
    c = ((imshape[1]/2)+35, 440)
    d = (imshape[1]-160,imshape[0])
'''''

#perspective transform
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)   
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

transformed_images = []
for i, image in enumerate(thresholded_images):
    imshape = image.shape
    a = (270,imshape[0]-50)
    b = ((imshape[1]/2)-125, 500)
    c = ((imshape[1]/2)+140, 500)
    d = (imshape[1]-240,imshape[0]-50)
    vertices = np.array([[a, b, c, d]], dtype=np.int32)
    src = np.array([[a, b, c, d]], dtype=np.float32)
    selected = region_of_interest(image, vertices)
    offsetx = 400
    offsety = 0
    dst = np.float32([[offsetx,imshape[0]-offsety], 
                      [offsetx,offsety], 
                      [imshape[1]-offsetx, offsety], 
                      [imshape[1]-offsetx, imshape[0]-offsety]])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(image, M, (imshape[1], imshape[0]), flags=cv2.INTER_LINEAR)
    transformed_images.append(warped)
    plt.figure()
    plt.imshow(images[i])
    plt.figure()
    plt.imshow(warped, cmap='gray')
    #plt.imshow(selected)
    #break
plt.show()